In [199]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Configuration de l'affichage
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
plt.style.use('seaborn-v0_8-darkgrid')

print("✅ Bibliothèques importées avec succès")

✅ Bibliothèques importées avec succès


In [200]:
file_path = './data/retail_store_inventory.csv'

# Chargement du dataset
df_raw = pd.read_csv(file_path, encoding='ISO-8859-1')

print("✅ Dataset chargé avec succès")
print(f"📦 Dimensions: {df_raw.shape[0]:,} lignes × {df_raw.shape[1]} colonnes")

✅ Dataset chargé avec succès
📦 Dimensions: 73,100 lignes × 15 colonnes


## Pretraitement des données

In [201]:
df_raw['Date'] = pd.to_datetime(df_raw['Date'])

### les valeurs manquantes

In [202]:
df_raw.isnull().sum()

Date                  0
Store ID              0
Product ID            0
Category              0
Region                0
Inventory Level       0
Units Sold            0
Units Ordered         0
Demand Forecast       0
Price                 0
Discount              0
Weather Condition     0
Holiday/Promotion     0
Competitor Pricing    0
Seasonality           0
dtype: int64

### Enrichisseement des variables temporelles

In [203]:
df_raw['week'] = df_raw['Date'].dt.isocalendar().week.astype(int)
df_raw['month'] = df_raw['Date'].dt.month
df_raw['quarter'] = df_raw['Date'].dt.quarter
df_raw['day_of_week'] = df_raw['Date'].dt.dayofweek
df_raw['is_weekend'] = (df_raw['day_of_week'] >= 5).astype(int)
df_raw['is_month_start'] = df_raw['Date'].dt.is_month_start.astype(int)
df_raw['is_month_end'] = df_raw['Date'].dt.is_month_end.astype(int)

In [204]:
print("PRÉVUE DES DONNÉES")

display(df_raw.head())

PRÉVUE DES DONNÉES


,Date,Store ID,Product ID,Category,Region,Inventory Level,Units Sold,Units Ordered,Demand Forecast,Price,Discount,Weather Condition,Holiday/Promotion,Competitor Pricing,Seasonality,week,month,quarter,day_of_week,is_weekend,is_month_start,is_month_end
0,2022-01-01,S001,P0001,Groceries,North,231,127,55,135.47,33.50,20,Rainy,0,29.69,Autumn,52,1,1,5,1,1,0
1,2022-01-01,S001,P0002,Toys,South,204,150,66,144.04,63.01,20,Sunny,0,66.16,Autumn,52,1,1,5,1,1,0
2,2022-01-01,S001,P0003,Toys,West,102,65,51,74.02,27.99,10,Sunny,1,31.32,Summer,52,1,1,5,1,1,0
3,2022-01-01,S001,P0004,Toys,North,469,61,164,62.18,32.72,10,Cloudy,1,34.74,Autumn,52,1,1,5,1,1,0
4,2022-01-01,S001,P0005,Electronics,East,166,14,135,9.26,73.64,0,Sunny,0,68.95,Summer,52,1,1,5,1,1,0


### Suppression des duplication

In [205]:
df_raw = df_raw.drop_duplicates()

### Ordre chronologique des données
Assure la cohérence chronologique indispensable pour créer des variables de retard et des statistiques glissantes.

In [206]:
df = df_raw.copy()
df = df.sort_values(['Store ID','Product ID','Date'])
df.head()

,Date,Store ID,Product ID,Category,Region,Inventory Level,Units Sold,Units Ordered,Demand Forecast,Price,Discount,Weather Condition,Holiday/Promotion,Competitor Pricing,Seasonality,week,month,quarter,day_of_week,is_weekend,is_month_start,is_month_end
0,2022-01-01,S001,P0001,Groceries,North,231,127,55,135.47,33.50,20,Rainy,0,29.69,Autumn,52,1,1,5,1,1,0
100,2022-01-02,S001,P0001,Groceries,West,116,81,104,92.94,27.95,10,Cloudy,0,30.89,Spring,52,1,1,6,1,0,0
200,2022-01-03,S001,P0001,Electronics,West,154,5,189,5.36,62.70,20,Rainy,0,58.22,Winter,1,1,1,0,0,0,0
300,2022-01-04,S001,P0001,Groceries,South,85,58,193,52.87,77.88,15,Cloudy,1,75.99,Winter,1,1,1,1,0,0,0
400,2022-01-05,S001,P0001,Groceries,South,238,147,37,150.27,28.46,20,Sunny,1,29.40,Winter,1,1,1,2,0,0,0


### Lag Features

**lag_1, lag_7, lag_14, lag_30** : Reflètent la mémoire du système : les ventes passées influencent directement les ventes futures.

Exemple : effet d’un pic de ventes après une promo ou une saisonnalité hebdomadaire.

In [207]:
lags = [1, 7, 14, 30]

for lag in lags:
    df[f'lag_{lag}'] = df.groupby(
        ['Store ID','Product ID']
    )['Units Sold'].shift(lag)

df.head()

,Date,Store ID,Product ID,Category,Region,Inventory Level,Units Sold,Units Ordered,Demand Forecast,Price,Discount,Weather Condition,Holiday/Promotion,Competitor Pricing,Seasonality,week,month,quarter,day_of_week,is_weekend,is_month_start,is_month_end,lag_1,lag_7,lag_14,lag_30
0,2022-01-01,S001,P0001,Groceries,North,231,127,55,135.47,33.50,20,Rainy,0,29.69,Autumn,52,1,1,5,1,1,0,NaN,NaN,NaN,NaN
100,2022-01-02,S001,P0001,Groceries,West,116,81,104,92.94,27.95,10,Cloudy,0,30.89,Spring,52,1,1,6,1,0,0,127.0,NaN,NaN,NaN
200,2022-01-03,S001,P0001,Electronics,West,154,5,189,5.36,62.70,20,Rainy,0,58.22,Winter,1,1,1,0,0,0,0,81.0,NaN,NaN,NaN
300,2022-01-04,S001,P0001,Groceries,South,85,58,193,52.87,77.88,15,Cloudy,1,75.99,Winter,1,1,1,1,0,0,0,5.0,NaN,NaN,NaN
400,2022-01-05,S001,P0001,Groceries,South,238,147,37,150.27,28.46,20,Sunny,1,29.40,Winter,1,1,1,2,0,0,0,58.0,NaN,NaN,NaN


### Rolling statistics

**Moyennes et écarts-types glissants (roll_mean, roll_std)**: Capturent les tendances locales et la volatilité des ventes.

Exemple : une hausse progressive des ventes sur 30 jours peut indiquer une tendance durable.

In [ ]:
windows = [7, 14, 30]

for w in windows:
    df[f'roll_mean_{w}'] = (
        df.groupby(['Store ID','Product ID'])['Units Sold']
          .transform(lambda x: x.shift(1).rolling(w).mean())  # ← Exclure la valeur actuelle
    )
    df[f'roll_std_{w}'] = (
        df.groupby(['Store ID','Product ID'])['Units Sold']
          .transform(lambda x: x.shift(1).rolling(w).std())
    )
    
df.head()

,Date,Store ID,Product ID,Category,Region,Inventory Level,Units Sold,Units Ordered,Demand Forecast,Price,Discount,Weather Condition,Holiday/Promotion,Competitor Pricing,Seasonality,week,month,quarter,day_of_week,is_weekend,is_month_start,is_month_end,lag_1,lag_7,lag_14,lag_30,roll_mean_7,roll_std_7,roll_mean_14,roll_std_14,roll_mean_30,roll_std_30
0,2022-01-01,S001,P0001,Groceries,North,231,127,55,135.47,33.50,20,Rainy,0,29.69,Autumn,52,1,1,5,1,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,2022-01-02,S001,P0001,Groceries,West,116,81,104,92.94,27.95,10,Cloudy,0,30.89,Spring,52,1,1,6,1,0,0,127.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200,2022-01-03,S001,P0001,Electronics,West,154,5,189,5.36,62.70,20,Rainy,0,58.22,Winter,1,1,1,0,0,0,0,81.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
300,2022-01-04,S001,P0001,Groceries,South,85,58,193,52.87,77.88,15,Cloudy,1,75.99,Winter,1,1,1,1,0,0,0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
400,2022-01-05,S001,P0001,Groceries,South,238,147,37,150.27,28.46,20,Sunny,1,29.40,Winter,1,1,1,2,0,0,0,58.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Price & Promotion Features
**price_after_discount, price_vs_competitor, promo_active**
Les prix et promotions sont des drivers directs de la demande.
Comparer au prix du concurrent est crucial pour modéliser l’élasticité.

In [209]:
df['price_after_discount'] = df['Price'] * (1 - df['Discount']/100)
df['price_vs_competitor'] = df['Price'] - df['Competitor Pricing']
df['promo_active'] = (df['Discount'] > 0).astype(int)


### Stock & Supply Features
**stock_ratio, order_to_sales**: Intègrent la dimension logistique : un stock faible ou des commandes excessives influencent les ventes.

In [210]:
df['stock_ratio'] = df['Inventory Level'] / (df['Units Sold'] + 1)
df['order_to_sales'] = df['Units Ordered'] / (df['Units Sold'] + 1)

### External Signals
**bad_weather, is_holiday** : Les conditions externes (météo, jours fériés) impactent fortement la demande.
Exemple : mauvais temps réduit les sorties, donc baisse des ventes en magasin.

In [211]:
df['bad_weather'] = df['Weather Condition'].isin(
    ['Rainy','Stormy','Snowy']
).astype(int)

df['is_holiday'] = df['Holiday/Promotion']


### Feature interactions
**promo_weekend, holiday_discount, season_promo**: Les interactions révèlent des effets combinés puissants (promo + week-end = explosion des ventes).


In [212]:
df['promo_weekend'] = df['promo_active'] * df['is_weekend']
df['holiday_discount'] = df['is_holiday'] * df['Discount']
df['season_promo'] = df['Seasonality'].astype(str) + "_" + df['promo_active'].astype(str)

df.head()

,Date,Store ID,Product ID,Category,Region,Inventory Level,Units Sold,Units Ordered,Demand Forecast,Price,Discount,Weather Condition,Holiday/Promotion,Competitor Pricing,Seasonality,week,month,quarter,day_of_week,is_weekend,is_month_start,is_month_end,lag_1,lag_7,lag_14,lag_30,roll_mean_7,roll_std_7,roll_mean_14,roll_std_14,roll_mean_30,roll_std_30,price_after_discount,price_vs_competitor,promo_active,stock_ratio,order_to_sales,bad_weather,is_holiday,promo_weekend,holiday_discount,season_promo
0,2022-01-01,S001,P0001,Groceries,North,231,127,55,135.47,33.50,20,Rainy,0,29.69,Autumn,52,1,1,5,1,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.800,3.81,1,1.804688,0.429688,1,0,1,0,Autumn_1
100,2022-01-02,S001,P0001,Groceries,West,116,81,104,92.94,27.95,10,Cloudy,0,30.89,Spring,52,1,1,6,1,0,0,127.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.155,-2.94,1,1.414634,1.268293,0,0,1,0,Spring_1
200,2022-01-03,S001,P0001,Electronics,West,154,5,189,5.36,62.70,20,Rainy,0,58.22,Winter,1,1,1,0,0,0,0,81.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.160,4.48,1,25.666667,31.500000,1,0,0,0,Winter_1
300,2022-01-04,S001,P0001,Groceries,South,85,58,193,52.87,77.88,15,Cloudy,1,75.99,Winter,1,1,1,1,0,0,0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66.198,1.89,1,1.440678,3.271186,0,1,0,15,Winter_1
400,2022-01-05,S001,P0001,Groceries,South,238,147,37,150.27,28.46,20,Sunny,1,29.40,Winter,1,1,1,2,0,0,0,58.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.768,-0.94,1,1.608108,0.250000,0,1,0,20,Winter_1


### Target Engineering (clé pour précision)
**target_7, target_30** 
- Permet de prédire la demande future (horizon de 7 ou 30 jours).
- Indispensable pour la planification logistique et les prévisions de stock.

**Suppression des NaN**:
 Les features de lag/rolling créent des valeurs manquantes au début des séries → il faut les retirer pour éviter les erreurs d’entraînement.

In [213]:
print(df.columns)

Index(['Date', 'Store ID', 'Product ID', 'Category', 'Region',
       'Inventory Level', 'Units Sold', 'Units Ordered', 'Demand Forecast',
       'Price', 'Discount', 'Weather Condition', 'Holiday/Promotion',
       'Competitor Pricing', 'Seasonality', 'week', 'month', 'quarter',
       'day_of_week', 'is_weekend', 'is_month_start', 'is_month_end', 'lag_1',
       'lag_7', 'lag_14', 'lag_30', 'roll_mean_7', 'roll_std_7',
       'roll_mean_14', 'roll_std_14', 'roll_mean_30', 'roll_std_30',
       'price_after_discount', 'price_vs_competitor', 'promo_active',
       'stock_ratio', 'order_to_sales', 'bad_weather', 'is_holiday',
       'promo_weekend', 'holiday_discount', 'season_promo'],
      dtype='object')


In [ ]:
df['target_7'] = df.groupby(['Store ID','Product ID'])['Units Sold'].shift(-7)
df['target_30'] = df.groupby(['Store ID','Product ID'])['Units Sold'].shift(-30)

df = df.dropna(subset=['lag_1', 'lag_7', 'roll_mean_7'])  # Exemple

In [215]:
df.dropna(inplace=True)

### Split temporel

In [217]:
df_boosting = df.copy()
train_df = df_boosting[df_boosting['Date'] < '2023-06-01']
test_df  = df_boosting[df_boosting['Date'] >= '2023-06-01']

### Suppression de collone inutile

In [218]:
cols_to_drop = [
    'Store ID',
    'Date',
    'Units Sold'
]

df_boosting = df_boosting.drop(columns=cols_to_drop)

In [ ]:
# Split temporel d'abord
train_df = df[df['Date'] < '2023-06-01']
test_df = df[df['Date'] >= '2023-06-01']

# Encodage séparé
from sklearn.preprocessing import OneHotEncoder

# Pour les variables catégorielles
cat_cols = ['Category', 'Region', 'Weather Condition', 'Seasonality', 'Product ID']

encoder = OneHotEncoder(drop='first', sparse_output=False)
train_encoded = encoder.fit_transform(train_df[cat_cols])
test_encoded = encoder.transform(test_df[cat_cols])

# Créer DataFrames avec les noms de colonnes
train_encoded_df = pd.DataFrame(
    train_encoded, 
    columns=encoder.get_feature_names_out(cat_cols),
    index=train_df.index
)
test_encoded_df = pd.DataFrame(
    test_encoded,
    columns=encoder.get_feature_names_out(cat_cols),
    index=test_df.index
)

# Fusionner
train_df = pd.concat([train_df.drop(columns=cat_cols), train_encoded_df], axis=1)
test_df = pd.concat([test_df.drop(columns=cat_cols), test_encoded_df], axis=1)

train_df.to_csv("./data/train_boosting.csv", index=False)
test_df.to_csv("./data/test_boosting.csv", index=False)

# Sauvegarde du DataFrame complet (si besoin)
df_boosting.to_csv("./data/dataset_boosting.csv", index=False)

## Pretraitement samira

In [220]:

df_sarimax = df_raw.drop(columns=[
    'Store ID',
    'Category',
    'Region',
    'Seasonality',
    'Weather Condition',
    'Inventory Level',
    'Units Ordered',
    'Demand Forecast'
], errors='ignore')

df_sarimax = (
    df_sarimax
    .groupby(['Product ID', 'Date'])
    .agg({
        'Units Sold': 'sum',
        'Price': 'mean',
        'Discount': 'mean',
        'Holiday/Promotion': 'max',
        'Competitor Pricing': 'mean'
    })
    .reset_index()
)

# Resampling hebdomadaire PAR PRODUIT
df_sarimax = (
    df_sarimax
    .set_index('Date')
    .groupby('Product ID')
    .resample('W')
    .agg({
        'Units Sold': 'sum',
        'Price': 'mean',
        'Discount': 'mean',
        'Holiday/Promotion': 'max',
        'Competitor Pricing': 'mean'
    })
    .reset_index()
)

# Valeurs manquantes
df_sarimax = df_sarimax.sort_values(['Product ID', 'Date'])
df_sarimax = df_sarimax.fillna(method='ffill')
df_sarimax.to_csv("./data/sarimax_dataset.csv", index=False)

split_date = df_sarimax['Date'].quantile(0.8)

# Séparation train / test
train_sarimax = df_sarimax[df_sarimax['Date'] < split_date]
test_sarimax  = df_sarimax[df_sarimax['Date'] >= split_date]

# --- SAUVEGARDE ---
train_sarimax.to_csv("./data/train_sarimax.csv", index=False)
test_sarimax.to_csv("./data/test_sarimax.csv", index=False)

print("Prétraitement et split SARIMAX terminés !")
print(f"Train shape: {train_sarimax.shape}, Test shape: {test_sarimax.shape}")

# Sauvegarde


Prétraitement et split SARIMAX terminés !
Train shape: (1680, 7), Test shape: (440, 7)
